# Tic Tac Toe

This is the implementation of tic-tac-toe game in Chialisp. As you can see this puzzle has no knowledge about the blockchain. To play this game on the Chia blockchain, we will wrap this (inner) puzzle with another (outer) puzzle.

## Code
### [tic-tac-toe.clsp](./code/tic-tac-toe.clsp)
```lisp
;   BOARD : current tic tac toe board state (curried)
;   V     : x or o to be played (curried)
;   pos   : position to be played
(mod (BOARD V pos)
    (include tic-tac-toe.clib)

    (defun play (new_board V)
        (list 
            (check-board new_board V)
            new_board
        )
    )

    ; 1. get new board
    ; 2. return the play result and new board
    (play (get-new-board BOARD V pos) V)
)
```

### [tic-tac-toe.clib](./code/tic-tac-toe.clib)
```lisp
    ;   if v won return v
    ;   else if board is still playable return 0 (playable)
    ;   else return -1 (not playable)
    (defun check-board (board v)
        (if (= (check-winning-board board v) 1)
            v ; v won
            (check-playable-board board) ; 0 or -1
        )
    )
```

In [1]:
%%bash
chia version
cdv --version
python --version

1.5.1.dev0
cdv, version 1.0.8
Python 3.10.5


In [2]:
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

from chia.types.blockchain_format.program import Program

In [3]:
sys.path.insert(0, "./code")
import tic_tac_toe

tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])

def sim_play(board, player, position):
    puzzle = tic_tac_toe.get_curried_puzzle(tic_tac_toe_puzzle, board, player)
    board_state, board = tic_tac_toe.play(puzzle, position)
    tic_tac_toe.print_board(board)
    print({
         0 : '',
        -1 : 'tie game',
        120: 'x wins',
        113: 'o wins'
    }[board_state])

## Python 3.10+
#    match board_state:
#        case -1:
#            print('tie game')
#        case 120:
#            print('x wins')
#        case 113:
#            print('o wins')

    player = 'x' if player == 'o' else 'o'
    return board, player



## Simulate Winning Board
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until `x` wins.

In [4]:
board, player = sim_play([' '] * 9, 'x', 4)
board, player = sim_play(board, player, 0)
board, player = sim_play(board, player, 3)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 5)

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


 o |   |   
---+---+---
   | x |   
---+---+---
   |   |   


 o |   |   
---+---+---
 x | x |   
---+---+---
   |   |   


 o | o |   
---+---+---
 x | x |   
---+---+---
   |   |   


 o | o |   
---+---+---
 x | x | x 
---+---+---
   |   |   

x wins


## Simulate Tie Game
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until the board is not playable.

In [5]:
board, player = sim_play([' '] * 9, 'x', 0)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 2)
board, player = sim_play(board, player, 4)
board, player = sim_play(board, player, 5)
board, player = sim_play(board, player, 6)
board, player = sim_play(board, player, 7)
board, player = sim_play(board, player, 8)
board, player = sim_play(board, player, 3)

 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   


 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   


 x | o | x 
---+---+---
   |   |   
---+---+---
   |   |   


 x | o | x 
---+---+---
   | o |   
---+---+---
   |   |   


 x | o | x 
---+---+---
   | o | x 
---+---+---
   |   |   


 x | o | x 
---+---+---
   | o | x 
---+---+---
 o |   |   


 x | o | x 
---+---+---
   | o | x 
---+---+---
 o | x |   


 x | o | x 
---+---+---
   | o | x 
---+---+---
 o | x | o 


 x | o | x 
---+---+---
 x | o | x 
---+---+---
 o | x | o 

tie game


## Simulate Errors

In [6]:
from clvm_tools.binutils import disassemble
try:
    board = sim_play([0] * 9, 'x', 9)
except ValueError as err:
    print(err)
    print(disassemble(Program.from_bytes(bytes.fromhex(err.args[1])).rest()))

('clvm raise', 'ff09ff90706f736974696f6e20696e76616c696480')
("position invalid")


In [7]:
try:
    board, player = sim_play([0] * 9, 'x', -1)
except ValueError as err:
    print(err)
    print(disassemble(Program.from_bytes(bytes.fromhex(err.args[1])).rest()))

('clvm raise', 'ff81ffff90706f736974696f6e20696e76616c696480')
("position invalid")


In [8]:
try:
    board, player = sim_play([0] * 9, 'x', 1)
    board, player = sim_play(board, player, 1)
except ValueError as err:
    print(err)
    print(disassemble(Program.from_bytes(bytes.fromhex(err.args[1])).rest()))

('clvm raise', 'ff01ff99706f736974696f6e20616c7265616479206f6363757069656480')
("position already occupied")


## Get Player & Board From Curried Puzzle


In [9]:
board = [' ', 'x', 'x', ' ', 'o', ' ', 'o', ' ', ' ']
player = 'x'
puzzle = tic_tac_toe.get_curried_puzzle(tic_tac_toe_puzzle, board, player)
board_from_puzzle = tic_tac_toe.get_board_from_curried_puzzle(puzzle)
player_from_puzzle = tic_tac_toe.get_player_from_curried_puzzle(puzzle)
tic_tac_toe.print_board(board_from_puzzle)
print(player_from_puzzle)

   | x | x 
---+---+---
   | o |   
---+---+---
 o |   |   

x


## Check Winning Before Check Tie Game

In [10]:
board = ['o', 'o', 'x', ' ', 'x', 'x', 'o', 'x', 'o']
player = 'x'
puzzle = tic_tac_toe.get_curried_puzzle(tic_tac_toe_puzzle, board, player)
board_from_puzzle = tic_tac_toe.get_board_from_curried_puzzle(puzzle)
player_from_puzzle = tic_tac_toe.get_player_from_curried_puzzle(puzzle)
tic_tac_toe.print_board(board_from_puzzle)
print(player_from_puzzle)
position = 3
board, player = sim_play(board, player, position)

 o | o | x 
---+---+---
   | x | x 
---+---+---
 o | x | o 

x
 o | o | x 
---+---+---
 x | x | x 
---+---+---
 o | x | o 

x wins


In [11]:
board = ['o', 'o', ' ', ' ', 'x', 'x', 'x', 'x', 'o']
player = 'x'
puzzle = tic_tac_toe.get_curried_puzzle(tic_tac_toe_puzzle, board, player)
board_from_puzzle = tic_tac_toe.get_board_from_curried_puzzle(puzzle)
player_from_puzzle = tic_tac_toe.get_player_from_curried_puzzle(puzzle)
tic_tac_toe.print_board(board_from_puzzle)
print(player_from_puzzle)
position = 2
board, player = sim_play(board, player, position)

 o | o |   
---+---+---
   | x | x 
---+---+---
 x | x | o 

x
 o | o | x 
---+---+---
   | x | x 
---+---+---
 x | x | o 

x wins


## Check Tie Game

In [12]:
board = ['x', 'o', 'x', ' ', 'o', 'x', 'o', 'x', 'o']
player = 'x'
puzzle = tic_tac_toe.get_curried_puzzle(tic_tac_toe_puzzle, board, player)
board_from_puzzle = tic_tac_toe.get_board_from_curried_puzzle(puzzle)
player_from_puzzle = tic_tac_toe.get_player_from_curried_puzzle(puzzle)
tic_tac_toe.print_board(board_from_puzzle)
print(player_from_puzzle)
position = 3
board, player = sim_play(board, player, position)

 x | o | x 
---+---+---
   | o | x 
---+---+---
 o | x | o 

x
 x | o | x 
---+---+---
 x | o | x 
---+---+---
 o | x | o 

tie game
